In [6]:
import mlflow
mlflow.set_tracking_uri("http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/")

In [7]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='s3://ms-mlflow-bucket/500246362129033379', creation_time=1751027568684, experiment_id='500246362129033379', last_update_time=1751027568684, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [9]:
df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

(36243, 2)

In [10]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for KNN

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 2000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)  # Tuning the number of neighbors
    p = trial.suggest_categorical('p', [1, 2])  # Tuning the distance metric (1 for Manhattan, 2 for Euclidean)

    # KNeighborsClassifier setup
    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow, passing the algo_name as "KNN"
    log_mlflow("KNN", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for KNN
run_optuna_experiment()


[I 2025-06-27 21:07:13,901] A new study created in memory with name: no-name-b1520b48-fbd1-4f84-99ed-b51c892ee3ff
[I 2025-06-27 21:07:21,964] Trial 0 finished with value: 0.49361702127659574 and parameters: {'n_neighbors': 24, 'p': 2}. Best is trial 0 with value: 0.49361702127659574.
[I 2025-06-27 21:07:28,839] Trial 1 finished with value: 0.5051063829787235 and parameters: {'n_neighbors': 20, 'p': 2}. Best is trial 1 with value: 0.5051063829787235.
[I 2025-06-27 21:07:36,091] Trial 2 finished with value: 0.5804255319148937 and parameters: {'n_neighbors': 3, 'p': 2}. Best is trial 2 with value: 0.5804255319148937.
[I 2025-06-27 21:07:42,036] Trial 3 finished with value: 0.3624468085106383 and parameters: {'n_neighbors': 12, 'p': 1}. Best is trial 2 with value: 0.5804255319148937.
[I 2025-06-27 21:07:48,078] Trial 4 finished with value: 0.3930851063829787 and parameters: {'n_neighbors': 4, 'p': 1}. Best is trial 2 with value: 0.5804255319148937.
[I 2025-06-27 21:07:53,986] Trial 5 finis

🏃 View run KNN_SMOTE_TFIDF_Trigrams at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/500246362129033379/runs/bccbdb5a82ba44ee876c3f27fbbde688
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/500246362129033379
